# Imports

In [1]:
import numpy as np
import tensorflow as tf
import os
import sys
sys.path.append('../../')
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import utils
tfk = tf.keras 
tfkl = tfk.layers

Init Plugin
Init Graph Optimizer
Init Kernel


# Setting Seed

In [2]:
seed = 17560

np.random.seed(seed)
tf.random.set_seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
tf.compat.v1.set_random_seed(seed)

In [3]:
training_dir = os.path.join('.', 'training')
validation_dir = os.path.join('.', 'validation')

# setting usefull parameters for training
batch_size = 16
epochs = 200
patience = 20
hiddens = 1
neurons = [32]
input_shape = (64, 64, 3)

# Model building

In [4]:
train_data_gen = ImageDataGenerator(rescale=1/255.)

valid_data_gen = ImageDataGenerator(rescale=1/255.)

train_gen = train_data_gen.flow_from_directory(directory=training_dir,
                                               target_size=input_shape[:2],
                                               interpolation='bilinear',
                                               color_mode='rgb',
                                               batch_size=batch_size,
                                               classes=None,
                                               shuffle=True,
                                               seed=seed)  

valid_gen = valid_data_gen.flow_from_directory(directory=validation_dir,
                                               target_size=input_shape[:2],
                                               interpolation='bilinear',
                                               color_mode='rgb',
                                               batch_size=batch_size,
                                               classes=None,
                                               shuffle=False,
                                               seed=seed)  
                                        

Found 14176 images belonging to 14 classes.
Found 3552 images belonging to 14 classes.


In [5]:
tl_model = utils.build_tl_vgg_model(hiddens=hiddens, neurons=neurons, input_shape=input_shape, seed=seed)

# Compile the model
tl_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics=['accuracy','categorical_accuracy', 'Precision', 'Recall'])
tl_model.summary()

Metal device set to: Apple M1 Pro


2021-11-28 17:55:21.744154: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-11-28 17:55:21.744278: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
vgg16 (Functional)           (None, 2, 2, 512)         14714688  
_________________________________________________________________
Flattening (Flatten)         (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 32)                65568     
_________________________________________________________________
dense_1 (Dense)              (None, 14)                462       
Total params: 14,780,718
Trainable params: 66,030
Non-trainable params: 14,714,688
____________________________________________

# Training and Result visualization

In [6]:
tl_history = tl_model.fit(
    x = train_gen,
    batch_size = batch_size,
    epochs = 1,
    validation_data = valid_gen,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=patience, restore_best_weights=True)]
).history

2021-11-28 17:55:24.670858: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-11-28 17:55:24.670995: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


  1/886 [..............................] - ETA: 7:15 - loss: 3.8100 - accuracy: 0.0000e+00 - categorical_accuracy: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00

2021-11-28 17:55:24.977482: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


884/886 [============================>.] - ETA: 0s - loss: 0.8306 - accuracy: 0.7484 - categorical_accuracy: 0.7484 - precision: 0.8862 - recall: 0.6268

2021-11-28 17:55:43.847853: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


886/886 [==============================] - 23s 26ms/step - loss: 0.8301 - accuracy: 0.7482 - categorical_accuracy: 0.7482 - precision: 0.8861 - recall: 0.6268 - val_loss: 0.4093 - val_accuracy: 0.8801 - val_categorical_accuracy: 0.8801 - val_precision: 0.9263 - val_recall: 0.8178


In [7]:
name = utils.get_model_name(hiddens=hiddens, neurons=neurons)
tl_model.save(name)

1h_32n


In [9]:
utils.plot_history(tl_history)

NameError: name 'tl_history' is not defined